下載資料套件

In [210]:
import urllib3
import requests
from bs4 import BeautifulSoup

處理資料套件

In [211]:
import pandas as pd
from datetime import datetime
from io import StringIO

載入其他檔案

In [212]:
from headers import *

畫圖套件

主程式碼

In [213]:
url = "https://www.taifex.com.tw/cht/3/futContractsDate"
#querydate = datetime.now().strftime("%Y/%m/%d")
querydate = "2024/08/26"
print(f"爬取日期: {querydate}")
params = {
        'queryType': '2',
        'goDay': '',
        'doQuery': '1',
        'dateaddcnt': '',
        'queryDate': querydate
    }

response = requests.post(url,headers=TAIFEX_OI,data=params)

if response.status_code == 200:
    dfs = pd.read_html(StringIO(response.text))
    if len(dfs) > 0:
        print(len(dfs))
        tdf = dfs[0]
        print("show")
        tdf
    
    else:
        print("沒有找到數據表格")
        
else:
    print(f"請求失敗，狀態碼：{response.status_code}")
        


爬取日期: 2024/08/26
1
show


In [214]:
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.table
table


<table border="0" bordercolordark="#ffffff" cellpadding="0" cellspacing="0" class="table_f table-sticky-3 w-1000" width="100%">
<thead>
<tr class="12bk">
<th align="middle" bgcolor="#6699CC" class="sheet-sticky serial-1 serial-all" colspan="3" rowspan="2" scope="colgroup"></th>
<th align="middle" bgcolor="#6699CC" colspan="6" scope="colgroup">交易口數與契約金額</th>
<th align="middle" bgcolor="#6699CC" colspan="6" scope="colgroup">未平倉餘額</th>
</tr>
<tr class="12bk">
<th align="right" bgcolor="#6699CC" colspan="2" scope="colgroup">多方</th>
<th align="right" bgcolor="#6699CC" colspan="2" scope="colgroup">空方</th>
<th align="right" bgcolor="#6699CC" colspan="2" scope="colgroup">多空淨額</th>
<th align="right" bgcolor="#6699CC" colspan="2" scope="colgroup">多方</th>
<th align="right" bgcolor="#6699CC" colspan="2" scope="colgroup">空方</th>
<th align="right" bgcolor="#6699CC" colspan="2" scope="colgroup">多空淨額</th>
</tr>
<tr class="12bk">
<th bgcolor="#6699CC" class="sheet-sticky serial-1" scope="col"><div alig

In [215]:
df = pd.read_html(StringIO(str(table)))
df

[   Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 交易口數與契約金額  \
    Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1        多方   
                   序 號              商品 名稱                身份別        口數   
 0                   1               臺股期貨                自營商      8703   
 1                   1               臺股期貨                 投信       940   
 2                   1               臺股期貨                 外資     76949   
 3                   2               電子期貨                自營商       113   
 4                   2               電子期貨                 投信         3   
 ..                ...                ...                ...       ...   
 65                 22          英國富時100期貨                 外資         3   
 66              期貨 小計              期貨 小計                自營商     75390   
 67              期貨 小計              期貨 小計                 投信       944   
 68              期貨 小計              期貨 小計                 外資    421197   
 69               期貨合計               期

In [216]:
print(len(df))
df = df[0]

1


In [217]:
#df.to_csv('test.csv', index=False, encoding='utf-8-sig')

In [218]:
df.drop(columns = df.columns[0],axis=1,inplace=True)
df.head()


Unnamed: 1_level_0 Unnamed: 2_level_0 交易口數與契約金額                    \
  Unnamed: 1_level_1 Unnamed: 2_level_1        多方                空方   
               商品 名稱                身份別        口數      契約 金額     口數   
0               臺股期貨                自營商      8703   38801532   9085   
1               臺股期貨                 投信       940    4174820    351   
2               臺股期貨                 外資     76949  342995500  78329   
3               電子期貨                自營商       113     539579    112   
4               電子期貨                 投信         3      14410     40   

                             未平倉餘額                                      \
              多空淨額              多方                空方              多空淨額   
       契約 金額    口數    契約 金額     口數      契約 金額     口數      契約 金額     口數   
0   40517488  -382 -1715955   6034   26801191   6016   26718995     18   
1    1558628   589  2616192  35940  159645480   9148   40635406  26792   
2  349213601 -1380 -6218101  20892   92801809  57881  257106751 -36989   
3     535570     1     4009    123     583608    166     787637    -43   
4     191358   -37  -176947    163     773402     10      47448    153   

              
              
       契約 金額  
0      82196  
1  119010074  
2 -164304942  
3    -204029  
4     725954

In [219]:
df.columns=['商品名稱','身份別','多方交易口數','多方交易金額','空方交易口數','空方交易金額','交易淨口數','交易淨金額','多方未平倉口數','多方未平倉金額','空方未平倉口數','空方未平倉金額','未平倉淨口數','未平倉淨金額']
df

,商品名稱,身份別,多方交易口數,多方交易金額,空方交易口數,空方交易金額,交易淨口數,交易淨金額,多方未平倉口數,多方未平倉金額,空方未平倉口數,空方未平倉金額,未平倉淨口數,未平倉淨金額
0,臺股期貨,自營商,8703,38801532,9085,40517488,-382,-1715955,6034,26801191,6016,26718995,18,82196
1,臺股期貨,投信,940,4174820,351,1558628,589,2616192,35940,159645480,9148,40635406,26792,119010074
2,臺股期貨,外資,76949,342995500,78329,349213601,-1380,-6218101,20892,92801809,57881,257106751,-36989,-164304942
3,電子期貨,自營商,113,539579,112,535570,1,4009,123,583608,166,787637,-43,-204029
4,電子期貨,投信,3,14410,40,191358,-37,-176947,163,773402,10,47448,153,725954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,英國富時100期貨,外資,3,1253,4,1675,-1,-422,0,0,23,9620,-23,-9620
66,期貨 小計,自營商,75390,63365423,68099,66032581,7291,-2667153,63714,45065945,256491,102845428,-192777,-57779483
67,期貨 小計,投信,944,4190340,393,1754122,551,2436219,38861,163665400,11423,44555662,27438,119109738
68,期貨 小計,外資,421197,567062028,443186,580061592,-21989,-12999568,129025,149947316,416917,392042857,-287892,-242095541


#篩選指定的期貨類型

In [220]:
df_filter = df.loc[df['商品名稱'].isin(['臺股期貨','小型臺指期貨','微型臺指期貨'])]
df_filter

,商品名稱,身份別,多方交易口數,多方交易金額,空方交易口數,空方交易金額,交易淨口數,交易淨金額,多方未平倉口數,多方未平倉金額,空方未平倉口數,空方未平倉金額,未平倉淨口數,未平倉淨金額
0,臺股期貨,自營商,8703,38801532,9085,40517488,-382,-1715955,6034,26801191,6016,26718995,18,82196
1,臺股期貨,投信,940,4174820,351,1558628,589,2616192,35940,159645480,9148,40635406,26792,119010074
2,臺股期貨,外資,76949,342995500,78329,349213601,-1380,-6218101,20892,92801809,57881,257106751,-36989,-164304942
9,小型臺指期貨,自營商,12480,13915573,13987,15589296,-1507,-1673723,2013,2235233,13295,14760816,-11282,-12525583
10,小型臺指期貨,投信,1,1110,0,0,1,1110,79,87730,15,16658,64,71072
11,小型臺指期貨,外資,149473,166589119,150321,167545302,-848,-956183,4970,5519164,4110,4564121,860,955043
12,微型臺指期貨,自營商,13233,2952337,11828,2634621,1405,317717,549,121921,9971,2214362,-9422,-2092441
13,微型臺指期貨,投信,0,0,0,0,0,0,0,0,0,0,0,0
14,微型臺指期貨,外資,42259,9421095,44230,9855950,-1971,-434856,2100,466410,9950,2209895,-7850,-1743485


#載入MongoDB套件

In [221]:
from pymongo import MongoClient
from pymongo.server_api import ServerApi
import json

In [222]:


# 從配置文件讀取連接信息
with open('config.json') as config_file:
    config = json.load(config_file)

mongo_user = config['mongo_user']
mongo_pass = config['mongo_pass']
mongo_host = config['mongo_host']

# 構建連接 URI
db_uri = f"mongodb+srv://{mongo_user}:{mongo_pass}@{mongo_host}/?retryWrites=true&w=majority&appName=FDB"

# 創建客戶端連接
client = MongoClient(db_uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)



Pinged your deployment. You successfully connected to MongoDB!


In [223]:
# 進行數據操作...
client = MongoClient(db_uri, server_api=ServerApi('1'))
db = client['findata']
collection = db['txf_oi']

#檢查是否已存在該日期數據
existing_data = collection.find_one({'date':querydate})
if existing_data:
    print(f"{querydate}的數據已存在,跳過存入")


data_dict = df_filter.to_dict('records')

for record in data_dict:
    record['date'] = datetime.strptime(querydate,"%Y/%m/%d")



result = collection.insert_many(data_dict)
print(f"成功插入 {len(result.inserted_ids)} 條文檔")

client.close()



成功插入 9 條文檔


In [224]:

# 可選：驗證插入的數據
# 重新連接並查詢數據
client = MongoClient(db_uri, server_api=ServerApi('1'))
db = client['findata']
collection = db['txf_oi']

# 查詢並打印前幾條數據
for doc in collection.find().limit(5):
    print(doc)


client.close()

{'_id': ObjectId('66d6adbcf41c948ee0603adb'), '商品名稱': '臺股期貨', '身份別': '自營商', '多方交易口數': 5479, '多方交易金額': 24446989, '空方交易口數': 6314, '空方交易金額': 28185513, '交易淨口數': -835, '交易淨金額': -3738525, '多方未平倉口數': 5504, '多方未平倉金額': 24566126, '空方未平倉口數': 6190, '空方未平倉金額': 27626595, '未平倉淨口數': -686, '未平倉淨金額': -3060469, '日期': '2024/08/30'}
{'_id': ObjectId('66d6adbcf41c948ee0603adc'), '商品名稱': '臺股期貨', '身份別': '投信', '多方交易口數': 186, '多方交易金額': 830092, '空方交易口數': 101, '空方交易金額': 450486, '交易淨口數': 85, '交易淨金額': 379606, '多方未平倉口數': 33911, '多方未平倉金額': 151365140, '空方未平倉口數': 8110, '空方未平倉金額': 36199792, '未平倉淨口數': 25801, '未平倉淨金額': 115165348, '日期': '2024/08/30'}
{'_id': ObjectId('66d6adbcf41c948ee0603add'), '商品名稱': '臺股期貨', '身份別': '外資', '多方交易口數': 65400, '多方交易金額': 291977999, '空方交易口數': 67983, '空方交易金額': 303504733, '交易淨口數': -2583, '交易淨金額': -11526734, '多方未平倉口數': 19235, '多方未平倉金額': 85857039, '空方未平倉口數': 58789, '空方未平倉金額': 262410037, '未平倉淨口數': -39554, '未平倉淨金額': -176552998, '日期': '2024/08/30'}
{'_id': ObjectId('66d6adbcf41c948ee0603ade'), '商品名稱':